In [1]:
import atlite
from shapely.geometry import box
import xarray as xr
import pandas as pd

In [2]:
# # Define your target lat/lon (e.g., Ras Al Khair, Saudi Arabia)
# lat = 27.5
# lon = 49.25

# # Define a small bounding box around the point (ERA5 resolution is ~0.25°)
# delta = 0.25  # half a grid cell
# # Create bounding box and extract bounds
# bounds_polygon = box(lon, lat, lon + delta, lat + delta)
# bounds = bounds_polygon.bounds

# # Create the cutout
# cutout = atlite.Cutout(
#     path="ras_al_khair_2018.nc",
#     module="era5",
#     bounds=bounds,
#     time="2018",
# )

# cutout.prepare(["influx"], monthly_requests=True)

In [ ]:
# Files and variable names
files = {
    'azimuth angle (deg)': 'azimuth.nc',
    'altitude angle (deg)': 'altitude.nc',
    'shortwave radiation downwards (W/m^2)': 'downward_shortwave.nc',
    'DNI (W/m^2)': 'dni.nc',
    "radiation hitting trough (Ken and Alicia's calculation) (W/m^2)": 'flux_dir_one_axis.nc',
}

# Target location
target_lat = 27.5
target_lon = 49.25

# Load the first dataset to get grid info
ref_var, ref_path = next(iter(files.items()))
ref_ds = xr.open_dataset(ref_path, engine="netcdf4")

# Find nearest (x, y) indices
y_index = abs(ref_ds.lat - target_lat).argmin().item()
x_index = abs(ref_ds.lon - target_lon).argmin().item()

# Extract time once
time = ref_ds.time.values
data = {'time (UTC)': time}

# Loop through each variable and extract values at location
for var_name, file_path in files.items():
    ds = xr.open_dataset(file_path, engine="netcdf4")
    data[var_name] = ds[list(ds.data_vars)[0]].isel(y=y_index, x=x_index).values

    # Translate azimuth and altitude angles from rad to degrees
    if var_name == 'azimuth angle (deg)':
        data[var_name] = data[var_name] * 180 / 3.14159
    elif var_name == 'altitude angle (deg)':
        data[var_name] = data[var_name] * 180 / 3.14159


# Create DataFrame and save
df = pd.DataFrame(data)
df.to_csv('az_alt_downRad_SaudiArabia.csv', index=False)

print("CSV file 'sun_data_timeseries.csv' has been created.")


CSV file 'sun_data_timeseries.csv' has been created.
